In [2]:
import tensorflow as tf
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [11]:
iris = load_iris()
iris_X, iris_y = iris.data[:-1,:], iris.target[:-1]
iris_y= pd.get_dummies(iris_y).values
trainX, testX, trainY, testY = train_test_split(iris_X, iris_y, test_size=0.33, random_state=42)

In [12]:
# numFeatures is the number of features in our input data.
# In the iris dataset, this number is '4'.
numFeatures = trainX.shape[1]

# numLabels is the number of classes our data points can be in.
# In the iris dataset, this number is '3'.
numLabels = trainY.shape[1]


# Placeholders
# 'None' means TensorFlow shouldn't expect a fixed number in that dimension
X = tf.placeholder(tf.float32, [None, numFeatures]) # Iris has 4 features, so X is a tensor to hold our data.
yGold = tf.placeholder(tf.float32, [None, numLabels]) # This will be our correct answers matrix for 3 classes.

In [13]:
W = tf.Variable(tf.zeros([4, 3]))  # 4-dimensional input and  3 classes
b = tf.Variable(tf.zeros([3])) # 3-dimensional output [0,0,1],[0,1,0],[1,0,0]

In [14]:
#Randomly sample from a normal distribution with standard deviation .01

weights = tf.Variable(tf.random_normal([numFeatures,numLabels],
                                       mean=0,
                                       stddev=0.01,
                                       name="weights"))

bias = tf.Variable(tf.random_normal([1,numLabels],
                                    mean=0,
                                    stddev=0.01,
                                    name="bias"))

In [15]:
# Three-component breakdown of the Logistic Regression equation.
# Note that these feed into each other.
apply_weights_OP = tf.matmul(X, weights, name="apply_weights")
add_bias_OP = tf.add(apply_weights_OP, bias, name="add_bias") 
activation_OP = tf.nn.sigmoid(add_bias_OP, name="activation")

# Number of Epochs in our training
numEpochs = 700

# Defining our learning rate iterations (decay)
learningRate = tf.train.exponential_decay(learning_rate=0.0008,
                                          global_step= 1,
                                          decay_steps=trainX.shape[0],
                                          decay_rate= 0.95,
                                          staircase=True)


#Defining our cost function - Squared Mean Error
cost_OP = tf.nn.l2_loss(activation_OP-yGold, name="squared_error_cost")

#Defining our Gradient Descent
training_OP = tf.train.GradientDescentOptimizer(learningRate).minimize(cost_OP)

In [16]:
# Create a tensorflow session
sess = tf.Session()

# Initialize our weights and biases variables.
init_OP = tf.global_variables_initializer()

# Initialize all tensorflow variables
sess.run(init_OP)

In [17]:
# argmax(activation_OP, 1) returns the label with the most probability
# argmax(yGold, 1) is the correct label
correct_predictions_OP = tf.equal(tf.argmax(activation_OP,1),tf.argmax(yGold,1))

# If every false prediction is 0 and every true prediction is 1, the average returns us the accuracy
accuracy_OP = tf.reduce_mean(tf.cast(correct_predictions_OP, "float"))

# Summary op for regression output
activation_summary_OP = tf.summary.histogram("output", activation_OP)

# Summary op for accuracy
accuracy_summary_OP = tf.summary.scalar("accuracy", accuracy_OP)

# Summary op for cost
cost_summary_OP = tf.summary.scalar("cost", cost_OP)

# Summary ops to check how variables (W, b) are updating after each iteration
weightSummary = tf.summary.histogram("weights", weights.eval(session=sess))
biasSummary = tf.summary.histogram("biases", bias.eval(session=sess))

# Merge all summaries
merged = tf.summary.merge([activation_summary_OP, accuracy_summary_OP, cost_summary_OP, weightSummary, biasSummary])

# Summary writer
writer = tf.summary.FileWriter("summary_logs", sess.graph)

In [19]:
# Initialize reporting variables
cost = 0
diff = 1
epoch_values = []
accuracy_values = []
cost_values = []

# Training epochs
for i in range(numEpochs):
    if i > 1 and diff < .0001:
        print("change in cost %g; convergence."%diff)
        break
    else:
        # Run training step
        step = sess.run(training_OP, feed_dict={X: trainX, yGold: trainY})
        # Report occasional stats
        if i % 10 == 0:
            # Add epoch to epoch_values
            epoch_values.append(i)
            # Generate accuracy stats on test data
            train_accuracy, newCost = sess.run([accuracy_OP, cost_OP], feed_dict={X: trainX, yGold: trainY})
            # Add accuracy to live graphing variable
            accuracy_values.append(train_accuracy)
            # Add cost to live graphing variable
            cost_values.append(newCost)
            # Re-assign values for variables
            diff = abs(newCost - cost)
            cost = newCost

            #generate print statements
            print("step %d, training accuracy %g, cost %g, change in cost %g"%(i, train_accuracy, newCost, diff))


# How well do we perform on held-out test data?
print("final accuracy on test set: %s" %str(sess.run(accuracy_OP, 
                                                     feed_dict={X: testX, 
                                                                yGold: testY})))

step 0, training accuracy 0.333333, cost 34.4502, change in cost 34.4502
step 10, training accuracy 0.50505, cost 30.279, change in cost 4.17118
step 20, training accuracy 0.646465, cost 28.3005, change in cost 1.97858
step 30, training accuracy 0.646465, cost 26.6301, change in cost 1.67031
step 40, training accuracy 0.646465, cost 25.2427, change in cost 1.38743
step 50, training accuracy 0.646465, cost 24.0972, change in cost 1.1455
step 60, training accuracy 0.646465, cost 23.1498, change in cost 0.947397
step 70, training accuracy 0.646465, cost 22.3613, change in cost 0.788548
step 80, training accuracy 0.646465, cost 21.6991, change in cost 0.662136
step 90, training accuracy 0.646465, cost 21.1376, change in cost 0.561518
step 100, training accuracy 0.656566, cost 20.6566, change in cost 0.481045
step 110, training accuracy 0.666667, cost 20.2403, change in cost 0.416225
step 120, training accuracy 0.666667, cost 19.8768, change in cost 0.363585
step 130, training accuracy 0.66

step 370, training accuracy 0.838384, cost 16.3589, change in cost 0.068264
step 380, training accuracy 0.838384, cost 16.2929, change in cost 0.0660439
step 390, training accuracy 0.838384, cost 16.2289, change in cost 0.0639668
step 400, training accuracy 0.848485, cost 16.1669, change in cost 0.062027
step 410, training accuracy 0.848485, cost 16.1067, change in cost 0.0602074
step 420, training accuracy 0.848485, cost 16.0482, change in cost 0.0584965
step 430, training accuracy 0.858586, cost 15.9913, change in cost 0.0568829
step 440, training accuracy 0.858586, cost 15.9359, change in cost 0.0553646
step 450, training accuracy 0.868687, cost 15.882, change in cost 0.0539284
step 460, training accuracy 0.878788, cost 15.8294, change in cost 0.0525646
step 470, training accuracy 0.878788, cost 15.7781, change in cost 0.0512772
step 480, training accuracy 0.878788, cost 15.7281, change in cost 0.0500517
step 490, training accuracy 0.878788, cost 15.6792, change in cost 0.0488834
st

step 560, training accuracy 0.89899, cost 15.3655, change in cost 0.0420666
step 570, training accuracy 0.89899, cost 15.3242, change in cost 0.0412464
step 580, training accuracy 0.89899, cost 15.2838, change in cost 0.0404596
step 590, training accuracy 0.909091, cost 15.2441, change in cost 0.0397015
step 600, training accuracy 0.909091, cost 15.2051, change in cost 0.03897
step 610, training accuracy 0.909091, cost 15.1668, change in cost 0.0382662
step 620, training accuracy 0.909091, cost 15.1293, change in cost 0.0375853
step 630, training accuracy 0.909091, cost 15.0923, change in cost 0.036931
step 640, training accuracy 0.909091, cost 15.056, change in cost 0.036293
step 650, training accuracy 0.909091, cost 15.0204, change in cost 0.0356808
step 660, training accuracy 0.909091, cost 14.9853, change in cost 0.0350876
step 670, training accuracy 0.909091, cost 14.9508, change in cost 0.0345106
step 680, training accuracy 0.909091, cost 14.9168, change in cost 0.0339527
step 69

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.plot([np.mean(cost_values[i-50:i]) for i in range(len(cost_values))])
plt.show()
